In this notebook, we'll be performing exploratory text analysis on the notes from Durham's CFS data.  Some ideas for places to take this:

 - Word frequencies for an initial look
 - Figure out the pattern (if any) behind the dispatcher scripts, including priority, chief complaint, questions/answers, and subject/vehicle descriptions

Some observations from reading remarks:
 - "x ft from STREET NAME" indicates directed patrol at that location
 - there is a structure for transports that includes the distance traveled with a transport, but the officers don't appear to fill it out consistently
 - 10 codes are seen as "10-72" or just "72"
 - CAD-generated messages are prefixed by a bracketed descriptor -- usually "[EPD]".  Messages left by the units are preceded by (UNIT NUMBER).
 - 

To look at word frequencies, we have to combine all the words into a single corpus.

In [3]:
import nltk
import dataset
from sqlalchemy import create_engine
import pandas as pd
from IPython.display import display

db_uri = "postgresql://jnance:@localhost:5432/cfs"

engine = create_engine(db_uri)
db = dataset.connect(db_uri)

/Users/jnance/anaconda/lib/python3.4/site-packages/sqlalchemy/dialects/postgresql/base.py:2421: SAWarning: Did not recognize type 'geometry' of column 'point'
  (attype, name))
/Users/jnance/anaconda/lib/python3.4/site-packages/sqlalchemy/dialects/postgresql/base.py:2421: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))
/Users/jnance/anaconda/lib/python3.4/site-packages/sqlalchemy/dialects/postgresql/base.py:2421: SAWarning: Did not recognize type 'geometry' of column 'extent'
  (attype, name))


In [2]:
notes = []

for row in db.query("SELECT body FROM note;"):
    if row['body']:
        notes += row['body'].lower().split()
    
txt = nltk.Text(notes)
fd = nltk.FreqDist(txt)

/Users/jnance/anaconda/lib/python3.4/site-packages/sqlalchemy/dialects/postgresql/base.py:2421: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))
/Users/jnance/anaconda/lib/python3.4/site-packages/sqlalchemy/dialects/postgresql/base.py:2421: SAWarning: Did not recognize type 'geometry' of column 'extent'
  (attype, name))


Here we can see the most frequent words in the corpus.  Many of these will probably become domain stopwords. (note: must download the nltk stopwords corpus using `nltk.download()` before this will work)

In [12]:
print("========= TOP 100 WORDS =========")
l = fd.most_common()
ndx = 0
num = 1
ignore = nltk.corpus.stopwords.words('english')
while (num <= 100 and ndx < len(l)):
    if l[ndx][0] not in ignore:
        print(str(num) + ". " + l[ndx][0])
        num += 1
    ndx += 1
print("\n")

========= TOP 100 WORDS =========
1. caller
2. [epd]
3. -
4. questions:
5. ***
6. scene.
7. is:
8. incident
9. vehicle
10. response:
11. chief
12. complaint:
13. statement:
14. 1.
15. 2.
16. 3.
17. 4.
18. 5.
19. 6.
20. info
21. /
22. 7.
23. one
24. code:
25. person
26. .
27. description
28. known
29. involved.
30. involves
31. dispatch
32. male
33. 8.
34. suspect
35. call
36. 9.
37. weapons
38. adv
39. 1.suspect:
40. back
41. 10.
42. location
43. race:
44. priority
45. gender:
46. 11.
47. color:
48. age:
49. danger.
50. clothing:
51. alarm
52. party
53. cb
54. happened
55. blk
56. business/resident/owner
57. suspect`s
58. reported
59. known.
60. 12.
61. phone
62. number
63. traffic
64. left
65. suspect/person
66. 2
67. progress.
68. area.
69. involved
70. female
71. victim
72. calling
73. suspicious
74. black
75. pd
76. name
77. responsible
78. n/a
79. make:
80. medical
81. mentioned.
82. area
83. needs
84. door
85. 13.
86. s/he
87. body:
88. [fire]
89. 2nd
90. victim.
91. disturbance


Many of these words are part of the call scripts transcribed by the CAD system, describing the caller's answers to specific questions asked by the dispatcher.  We'll probably need to add the script words to a stopword list, but we can keep the descriptive words that go along with them.

In [14]:
dpd_stopwords = set(nltk.corpus.stopwords.words('english'))

domain_stopwords = set(['caller', '[epd]', '-', 'questions:', 'scene.', '***', 'is:', 'incident', 'response:', 'chief',
                       'complaint:', 'statement:', 'info', 'description', 'known', 'involved', 'involves', 'dispatch',
                       '1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.', '11.', '12.', '13.', '/', 'code:',
                       '.', 'call', 'suspect', 'adv', '1.suspect:', 'location', 'race:', 'priority', 'gender:',
                       'color:', 'age:', 'clothing:', 'cb', 'suspect`s', 'reported', 'known.', 'suspect/person',
                       'progress.', 'area.', 'pd', 'name', 'n/a', 'make:', 'area', 's/he',
                       '[fire]', 'model:'])

dpd_stopwords.update(domain_stopwords)

Trying again with domain stopwords added:

In [16]:
print("========= TOP 100 WORDS =========")
l = fd.most_common()
ndx = 0
num = 1
ignore = dpd_stopwords
while (num <= 100 and ndx < len(l)):
    if l[ndx][0] not in ignore:
        print(str(num) + ". " + l[ndx][0])
        num += 1
    ndx += 1
print("\n")

========= TOP 100 WORDS =========
1. vehicle
2. one
3. person
4. involved.
5. male
6. weapons
7. back
8. danger.
9. alarm
10. party
11. happened
12. blk
13. business/resident/owner
14. phone
15. number
16. traffic
17. left
18. 2
19. female
20. victim
21. calling
22. suspicious
23. black
24. responsible
25. medical
26. mentioned.
27. needs
28. door
29. body:
30. 2nd
31. victim.
32. disturbance
33. see
34. ft
35. disturbance.
36. white
37. veh
38. line
39. open
40. theft
41. attention.
42. past:
43. aborted
44. leave
45. alarms
46. reportedly
47. caller.
48. 14.
49. drugs
50. monitoring
51. company.
52. two
53. activation
54. still
55. keyholder/owner
56. blocking
57. emergency
58. car
59. alarm.
60. law
61. incident.
62. contacted.
63. property
64. st
65. someone
66. front
67. going
68. advised
69. house
70. vm
71. suspect/person/vehicle
72. 1
73. 4
74. vehicles
75. blue
76. burglary/intrusion
77. tty
78. unk
79. no:
80. alcohol
81. 3
82. officer
83. vehicle.
84. people
85. 3rd
86. migh

#Gang-related
We did a segment for Tampa on gang-related calls.  In this case, we have the benefit of a "curated" data set, as the incidents have a flag indicating whether a given call is gang-related.  Let's see if we can classify calls as gang-related or not based on the call notes.

First, we'll get the info for each call.  Some of this may be useful in the classifier.

In [7]:
call_info = pd.read_sql("SELECT call.*, gang_related FROM call, "
                        "incident WHERE call.incident_id = incident.incident_id;",
                        engine, index_col="call_id")
call_info.head()

,month_received,week_received,dow_received,hour_received,case_id,call_source_id,primary_unit_id,first_dispatched_id,reporting_unit_id,street_num,...,first_unit_dispatch,first_unit_enroute,first_unit_arrive,first_unit_transport,last_unit_clear,time_closed,close_code_id,close_comments,incident_id,gang_related
call_id,,,,,,,,,,,,,,,,,,,,,
2014000003,1,1,2,0,14000001,8,162,162,162,1610,...,2014-01-01 00:00:41,NaT,2014-01-01 00:00:41,None,2014-01-01 00:15:57,2014-01-01 00:15:59,1,None,488517,False
2014000042,1,1,2,0,14000006,12,165,165,165,531,...,2014-01-01 00:20:01,2014-01-01 00:20:01,2014-01-01 00:29:39,None,2014-01-01 00:33:59,2014-01-01 00:34:01,1,None,488519,None
2014000083,1,1,2,0,14000009,3,9,9,9,4600,...,2014-01-01 00:54:21,2014-01-01 00:56:03,2014-01-01 01:02:43,None,2014-01-01 02:21:17,2014-01-01 02:21:17,1,None,488524,False
2014000140,1,1,2,1,14000013,3,353,353,353,1113,...,2014-01-01 01:35:55,2014-01-01 01:36:24,2014-01-01 01:37:13,2014-01-01 02:16:49,2014-01-01 03:11:03,2014-01-01 03:11:04,1,None,488525,None
2014000109,1,1,2,1,14000011,3,431,627,431,607,...,2014-01-01 01:13:52,2014-01-01 01:14:04,2014-01-01 01:24:30,2014-01-01 01:50:16,2014-01-01 03:21:13,2014-01-01 03:21:13,1,None,488526,False


Check out the columns to see which appear to be the most informative -- we'll drop any ones that are likely unnecessary or won't be accessible at the start of a call.

In [8]:
print(call_info.columns)

Index(['month_received', 'week_received', 'dow_received', 'hour_received', 'case_id', 'call_source_id', 'primary_unit_id', 'first_dispatched_id', 'reporting_unit_id', 'street_num', 'street_name', 'city_id', 'zip', 'crossroad1', 'crossroad2', 'geox', 'geoy', 'beat', 'district', 'sector', 'business', 'nature_id', 'priority', 'report_only', 'cancelled', 'time_received', 'time_routed', 'time_finished', 'first_unit_dispatch', 'first_unit_enroute', 'first_unit_arrive', 'first_unit_transport', 'last_unit_clear', 'time_closed', 'close_code_id', 'close_comments', 'incident_id', 'gang_related'], dtype='object')


In [9]:
drop_cols = ['case_id', 'reporting_unit_id', 'geox', 'geoy', 'report_only', 'cancelled', 'time_received',
             'time_routed', 'time_finished', 'first_unit_dispatch', 'first_unit_enroute', 'first_unit_arrive',
             'first_unit_transport', 'last_unit_clear', 'time_closed', 'close_code_id', 'close_comments']
for col in drop_cols:
    call_info.drop(col, axis=1)

In [ ]:
notes = pd.read_sql("SELECT * FROM note, call WHERE note.call_id = call.call_id AND call.incident_id IS NOT NULL;"):
    notes['call_id'].append(row['call_id'])